# AES-128 Key Search Using Grover's Algorithm on IQM Quantum Computer

This notebook demonstrates a conceptual hybrid quantum-classical approach to attempt decryption of AES-128 encrypted data using Grover's algorithm, executed on an IQM quantum computer.

## Overview
- AES-128 is a symmetric encryption algorithm with 128-bit key size.
- Grover's algorithm provides a quadratic speedup for unstructured search problems, such as brute forcing an AES key.
- The approach here is a proof-of-concept of implementing Grover's algorithm for AES key space search, where the oracle verifies if the key candidate decrypts ciphertext correctly.
- Actual implementation on current hardware is limited by qubit count and coherence times.

## Prerequisites
- Access to IQM quantum computer via IQMProvider.
- Python packages: `iqm-client[qiskit]`, `qiskit`, `pycryptodome` for AES operations.
- Encrypted data and partial known plaintext to verify candidates.


In [ ]:
# Install necessary packages (uncomment to run in your environment)
# !pip install "iqm-client[qiskit]" qiskit pycryptodome

## Setup IQM Quantum Computer Provider
Authenticate to IQM Resonance and get the backend to run quantum circuits.

In [ ]:
from iqm.qiskit_iqm import IQMProvider
import os
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import GroverOperator

# Set your IQM Resonance API token here or use environment variable IQM_API_TOKEN
# os.environ['IQM_API_TOKEN'] = 'your_api_token_here'

# Connect to IQM
provider = IQMProvider(url="https://cocos.resonance.meetiqm.com/garnet")
backend = provider.get_backend()

print("Connected to IQM backend:", backend.name())

## AES encryption and decryption helper
We use `pycryptodome` library to perform AES encryption and decryption on classical side to verify candidate keys.

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import binascii

def aes_encrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    return ciphertext

def aes_decrypt(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

# Example
key = b'1234567890abcdef'  # 16 bytes key
plaintext = b'Test data for AES'
ct = aes_encrypt(key, plaintext)
print("Ciphertext (hex):", binascii.hexlify(ct))
print("Decrypted text:", aes_decrypt(key, ct))

## Grover's Algorithm Oracle
The oracle function marks the correct AES key candidate. Due to complexity, here we demonstrate a small-scale placeholder oracle.

In realistic settings, implementing full AES oracle requires large qubit count and complex circuits which are beyond current hardware capabilities.

In [ ]:
# Placeholder oracle circuit for Grover's algorithm
def create_oracle(num_qubits):
    oracle = QuantumCircuit(num_qubits)
    # This is a dummy oracle that marks |11...1> state
    oracle.cz(0, num_qubits-1)
    return oracle.to_gate(label="Oracle")

# Number of qubits for Grover search (reduce for practical reason)
num_qubits = 4  # For demonstration, real AES key needs 128 qubits
oracle = create_oracle(num_qubits)
oracle.draw('mpl')

## Grover's Algorithm Circuit
Construct Grover operator and circuit with oracle and diffusion operator.

In [ ]:
from qiskit.circuit.library import GroverOperator

grover_op = GroverOperator(oracle)
grover_circuit = QuantumCircuit(num_qubits)
grover_circuit.h(range(num_qubits))  # Initialize superposition
grover_circuit.append(grover_op, range(num_qubits))
grover_circuit.measure_all()
grover_circuit.draw('mpl')

## Run on IQM or Simulator
Due to hardware constraints, start by running on a local simulator.
Switch to IQM backend when ready.

In [ ]:
backend_sim = Aer.get_backend('qasm_simulator')
job = execute(grover_circuit, backend_sim, shots=1024)
result = job.result()
counts = result.get_counts()
print("Simulation result counts:", counts)

## Integration with Encrypted Data
This part demonstrates how to verify candidate keys classically by decrypting the ciphertext and checking for correctness.

Replace `candidate_key` with keys decoded from measurement output.

Example below shows classical verification for a candidate key.

In [ ]:
def verify_key(candidate_key_bytes, ciphertext, known_plaintext):
    try:
        plaintext = aes_decrypt(candidate_key_bytes, ciphertext)
        # Check known pattern or full plaintext match
        return plaintext.startswith(known_plaintext)
    except Exception as e:
        return False

# Example use with known ciphertext and plaintext
known_plaintext = b'Test data'
example_ciphertext = ct  # previously generated ciphertext
candidate_key = b'1234567890abcdef'
print("Key valid?", verify_key(candidate_key, example_ciphertext, known_plaintext))

## Next Steps and Practical Considerations
- Full AES-128 Grover search requires ~128 qubits, very complex oracles.
- Current hardware like IQM Garnet supports fewer qubits but this notebook shows the conceptual framework.
- You can extend the oracle to implement AES logic using reversible circuits if resources permit.
- Hybrid quantum-classical loop needed: quantum to propose candidate keys, classical to verify decrypted correctness.
- Use the decrypted correctness check against the known plaintext or file format to identify the right key.
